# Data wrangling routes and charging stations
### *Assumes data is in a folder called RAWDATA in same folder

-----------------------------------------
TABULAR DATA-SPECIFIC INFORMATION FOR: 1-B. BusRoutes_UTA.dbf
-----------------------------------------


1. Number of variables:
5

2. Number of cases/rows: 
114

3. Variable List
    A. Name: LineAbbr
       Description: Abbreviations for bus lines.
	Nominal 
                    
    B. Name: LineName
       Description: Names of bus lines.
	Nominal

    C. Name: Service
       Description: Category of service of bus lines.
	Nominal, including local, express, BRT, fast bus, shuttle, ski, flex.

    D. Name: Frequency
       Description: Frequency of bus lines.
	Ratio

    E. Name: Shape_Leng
       Description: Length of the bus routes.
	Ratio, unit: foot

In [15]:
#read routes info and add to array -old
from dbfread import DBF
i = 0
routes = []
for record in DBF('RAWDATA\\1. Network Data\\1. BusRoutes_UTA\\BusRoutes_UTA.dbf'):
    i += 1
    routes.append({'lineAbbr': record['LineAbbr'], 'lineName': record["LineName"], 'service': record["Service"], 'shape_length': record["Shape_Leng"], 'busses': set()})

In [2]:
import json
#better file that has coordinates too
data = json.load(open("RAWDATA\\1. Network Data\\1. BusRoutes_UTA\\BusRoutes_UTA.json"))
routes = []
for r in data['features']:
    record = r["properties"]
    routes.append(
        {  
            'lineAbbr': record['LineAbbr'], 
            'lineName': record["LineName"], 
            'service': record["Service"], 
            'shape_length': record["Shape_Leng"], 
            'busses': set(),
            'coordinates': r['geometry']['coordinates']
        })


In [3]:
#calc path length
import math
for r in routes:
    for i in range(len(r['coordinates'])):
        sumdist = 0
        #calc dist between points 
        dist0 = r['coordinates'][i][0][0] - r['coordinates'][i - 1][0][0]
        dist1 = r['coordinates'][i][0][1] - r['coordinates'][i - 1][0][1]

        dist0 = r['coordinates'][i - 1][0][0] - r['coordinates'][i][0][0]
        dist1 = r['coordinates'][i - 1][0][1] - r['coordinates'][i][0][1]
        #sum of total distance
        sumdist += math.sqrt(dist0 * dist0 + dist1 * dist1)
    r['path_length'] = sumdist

In [4]:
print(routes[0]['path_length'])

0.0007600350806313509


In [19]:
#import list of busses from runcut
import openpyxl

wb_obj = openpyxl.load_workbook("RAWDATA\\1. Network Data\\3. UTA Runcut File  Aug2016.xlsx")
sheet = wb_obj.active

i = 0
for row in sheet.iter_rows():
    i += 1
    if i == 1:
        continue
    for route in routes:
        if route['lineAbbr'] == row[0].value:
            route['busses'].add(row[3].value)
    # if row[0].value in routes:
    #     routes[row[0].value]["busses"].add(row[3].value)



In [20]:
#write it to allRoutes.json
import json
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

with open('allRoutes.json', 'w') as outfile:
    json.dump(routes, outfile, default=set_default)

-----------------------------------------
TABULAR DATA-SPECIFIC INFORMATION FOR: 2-B. BusStops_UTA.dbf
-----------------------------------------


1. Number of variables:
4

2. Number of cases/rows: 
5987

3. Variable List
    A. Name: StopId
       Description: Unique identifier for bus stops.
	Nominal 
                    
    B. Name: StopName
       Description: The comlete names of bus stops.
	Nominal

    C. Name: StreetNum
       Description: The number of the streets the stops are on.
	Nominal

    D. Name: OnStreet
       Description: The name of the streets the stops are on.
	Nominal


In [20]:
#read in stops info from dbf file
stops = []
for record in DBF('RAWDATA\\1. Network Data\\2. BusStops_UTA\\BusStops_UTA.dbf'):
    stops.append({
        'StopId' : record['StopId'],
        'StopName': record["StopName"],
        'StreetNum': record["StreetNum"],
        'OnStreet': record["OnStreet"],
        'AtStreet': record["AtStreet"],
        'City': record["City"],
        'InService': record["InService"],
        'Bench': record["Bench"],
        'Shelter': record["Shelter"],
        'Lighting': record["Lighting"],
        'Garbage': record["Garbage"],
        'Bicycle': record["Bicycle"],
        'Transfer': record["Transfer"],
        'LocationUs': record["LocationUs"],
        'UTAStopID': record["UTAStopID"],
    })
    

In [1]:
import json
# better file that has properties and coordinates
stops = []
data = json.load(open("RAWDATA\\1. Network Data\\2. BusStops_UTA\BusStops_UTA.json"))
i = 0
for record in data["features"]:
    props = record["properties"]
    stops.append({
        'stopId' : props['StopId'],
        'stopName': props["StopName"],
        'streetNum': props["StreetNum"],
        'onStreet': props["OnStreet"],
        'atStreet': props["AtStreet"],
        'city': props["City"],
        'inService': props["InService"],
        'bench': props["Bench"],
        'shelter': props["Shelter"],
        'lighting': props["Lighting"],
        'garbage': props["Garbage"],
        'bicycle': props["Bicycle"],
        'transfer': props["Transfer"],
        'locationUs': props["LocationUs"],
        'UTAStopID': props["UTAStopID"],

        'coordinates': record['geometry']['coordinates']
    })
print(len(stops))

5987


In [40]:
#write to allStops
with open('allStops.json', 'w') as outfile:
    json.dump(stops, outfile)

### Don't think I did anything below here

In [33]:
plan = 'p180'
data = json.load(open("DATA\\plans\\" + plan + ".json"))
stops = json.load(open("allStopsNew.json"))
finalFile = "DATA\\stationLocations\\" + plan + ".json"
statLocs = {'type': 'FeatureCollection', 'features': []}
for cs in data['charging_stations']:
    coords = []
    # find allstopps where name matches
    for s in stops:
        if s["stopName"].replace(" ", "") == cs["stop_name"].replace(" ", ""):
            coords = s["coordinates"]
    if coords == []:
        print("NO COORDS", cs)
    geo = {'type': 'Point', 'coordinates': coords}
    ob = {'type': 'Feature', 'geometry': geo, 'properties': cs}
    statLocs['features'].append(ob)

with open(finalFile, 'w') as outfile:
    json.dump(statLocs, outfile)

In [97]:
import datetime

t = datetime.time(1, 2, 3)

plan = "p20"
stations = {}
dataBuses = json.load(open("DATA\\buses\\" + plan + ".json"))
# dataStations = json.load(open("DATA\\plans\\" + plan + ".json"))['charging_stations']
dataStations = json.load(open("allStopsNew.json"))

# for station in dataStations:
#     stations[station['stopName']] = {'stop_id': station['stopId'], 'stop_name': station['stopName'], 'busTimes': {}}
for i in range(24):
    timeStr = str(i) + ":00"
    time = datetime.time(i)
    for bus in dataBuses['buses']:
        for stp in bus['stops']:
            arvtime = datetime.time(0) if (stp['arrival_time'] == '') else datetime.time(int(stp['arrival_time'].split(":")[0]), int(stp['arrival_time'].split(":")[1]))
            deptime = datetime.time(23,59) if (stp['departure_time'] == '') else datetime.time(int(stp['departure_time'].split(":")[0]), int(stp['departure_time'].split(":")[1]))
            if arvtime <= time and time <= deptime: #at stop
                if stp['stop_name'] not in stations:
                    stations[stp['stop_name']] = {'stop_id': station['stopId'], 'stop_name': station['stopName'], 'busTimes': {}}
                if timeStr not in stations[stp['stop_name']]['busTimes']:
                    stations[stp['stop_name']]['busTimes'][timeStr] = []
                stations[stp['stop_name']]['busTimes'][timeStr].append(bus['id'])

with open('busesAtStations', 'w') as outfile:
    json.dump(stations, outfile)


In [96]:
# Stations
# [stationName : {'stop_id': x, 'stop_name': y, 'busTimes': {'0:00': [1,2,3], '1:00': [4,5,6] ... '23:00': [2,5,6]}}]
print(stations['FREEDOM BLVD @ 610 S'])

{'stop_id': 24302, 'stop_name': 'WASATCH BLVD @ 8551 S', 'busTimes': {'0:00': ['2049', '2001', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '1:00': ['2049', '2001', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '2:00': ['2049', '2001', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '3:00': ['2049', '2001', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '4:00': ['2049', '2001', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '5:00': ['2049', '2002', '2011', '2012', '2017', '2021', '2027', '2030', '2037', '2045', '2047'], '6:00': ['2049', '2000', '2006', '2027', '2030', '2037', '2045', '2047'], '7:00': ['2049', '2016', '2028', '2045', '2047'], '8:00': ['2049', '2001', '2013'], '9:00': ['2049', '2012', '2021'], '10:00': ['2049', '2016', '2018'], '11:00': ['2049', '2001', '2038'], '12:00': ['2049', '2002', '2021'], '13:0